In [18]:
#!pip3 install tensorflow -q

In [19]:
import pandas as pd
import numpy as np
import json
from tensorflow import keras
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
x_src_path = '../Dataset/assignment1.json'
y_src_path = '../DataBook/Assignment1_Data_Analyst.xlsx'

In [21]:
data = pd.read_json(x_src_path)


In [22]:
df_supervision = pd.read_excel(y_src_path)
plagiarised_array = df_supervision['Plagiarised'].astype(int).values

In [23]:
X = data.values
# Replace None with a default value (e.g., [0.0, 0.0, 0.0])
X = [[[0.0, 0.0, 0.0] if item is None else item for item in sublist] for sublist in X]

# Convert the list to a NumPy array
X_array = np.array(X, dtype=object)

# Determine the maximum length of the lists within the array
max_len = max(len(sublist) for sublist in X_array)

# Pad the lists with zeros to the maximum length
X_padded = np.array([np.pad(sublist, (0, max_len - len(sublist)), 'constant', constant_values=(0.0, 0.0)) for sublist in X_array])
X_padded = X_padded.astype(int)

OverflowError: Python int too large to convert to C long

In [ ]:
X = X_padded
y = plagiarised_array

In [ ]:
X

array([[[1706560727503, 0.0, 0.0],
        [1706560730105, 0.0, 0.0],
        [1706560730129, 0.0, 0.0],
        ...,
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0]],

       [[1706560190537, 0.0, 0.0],
        [1706560212575, 0.0, 0.0],
        [1706560217012, 0.0, 0.0],
        ...,
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0]],

       [[1706992495964, 0.0, 0.0],
        [1706993686727, 0.0, 0.0],
        [1706993689123, 0.0, 0.0],
        ...,
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0]],

       ...,

       [[1706560179936, 0.0, 0.0],
        [1706560182868, 0.0, 0.0],
        [1706560201909, 0.0, 0.0],
        ...,
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0]],

       [[1706726265694, 0.0, 0.0],
        [1706726269767, 0.0, 0.0],
        [1706726269791, 1.0, 1.0],
        ...,
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0]],

       [[170

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
x_train_flat = X_train.reshape(X_train.shape[0], -1)
x_test_flat = X_test.reshape(X_test.shape[0], -1)

In [ ]:
X_numeric = x_train_flat.astype(float)

# Check for NaN values
has_nan = np.isnan(X_numeric).any()

print(has_nan)

True


In [ ]:
perceptron = Perceptron()
perceptron.fit(x_train_flat, y_train)

# print train and test accuracy scores
print(f"Training data score: {perceptron.score(x_train_flat, y_train)}")
print(f"Training data score: {perceptron.score(x_test_flat, y_test)}")

In [ ]:
# let's use tensorflow keras library to build a 10 perceptron model
#  Dense layers are standard fully-interconnected neural network layers
model = keras.models.Sequential([
    keras.layers.Dense(10, activation='softmax') # output layer
])

# we'll use sparse_categorical_crossentropy as the loss function
LOSS_FN = keras.losses.sparse_categorical_crossentropy

# compile the model with standard backprop training algorithm called 'adam'
model.compile(optimizer='adam',loss=LOSS_FN,metrics=['accuracy'])

# train on training data, and validate on test data
# we'll train for 5 epochs
model.fit(x_train_flat, y_train, epochs=5, validation_data=(x_test_flat, y_test))



Epoch 1/5


2/2 [==============================] - 1s 142ms/step - loss: 1732.3601 - accuracy: 0.2326 - val_loss: 2027.1952 - val_accuracy: 0.3636
Epoch 2/5
2/2 [==============================] - 0s 31ms/step - loss: 1189.7917 - accuracy: 0.4651 - val_loss: 836.0984 - val_accuracy: 0.9091
Epoch 3/5
2/2 [==============================] - 0s 26ms/step - loss: 997.0189 - accuracy: 0.9070 - val_loss: 1361.5657 - val_accuracy: 0.9091
Epoch 4/5
2/2 [==============================] - 0s 25ms/step - loss: 1631.8879 - accuracy: 0.9070 - val_loss: 1633.9438 - val_accuracy: 0.9091
Epoch 5/5
2/2 [==============================] - 0s 28ms/step - loss: 1999.9896 - accuracy: 0.8837 - val_loss: 1729.4320 - val_accuracy: 0.9091


In [ ]:
# let's build a whole bunch of models to compare
modelNames = [
    '1d',
    '2d',
    '3d',
    '1c1d',
    '1c2d',
    '2c2d',
]

# in all models we'll have the same number of nodes per hidden layer
NODES_PER_HIDDEN_LAYER = 15

models = [
    # just the output layer
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 1 hidden layer
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 2 hidden layers
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),   # hidden layer 2
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 1 convolutional layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")  # output layer
    ]),
    # 1 convolutional layer and 1 hidden dense layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Flatten(),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation="softmax")  # output layer
    ]),
    # 2 convolutional layers and 1 hidden dense layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 2
        keras.layers.Flatten(),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation="softmax")  # output layer
    ])
]



In [ ]:
# compile all models
for model in models:
    model.compile(optimizer='adam',loss=LOSS_FN,metrics=['accuracy'])

In [ ]:
TRAINING_EPOCHS = 5

# train all models
for model, name in zip(models, modelNames):
    print(f'training model {name}')
    model.fit(X_train, y_train, epochs=TRAINING_EPOCHS)


training model 1d
Epoch 1/5


ValueError: in user code:

    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File

In [ ]:
# get all model accuracy scores on test data
scores = [model.evaluate(X_test,y_test)[1] for model in models]


In [ ]:
# let's also add the perceptron score to our accuracy scores list
modelNames.insert(0, 'perceptron')
scores.insert(0, perceptron.score(x_test_flat, y_test) )

# print all models and scores
for name,score in zip(modelNames,scores):
    print(name, score)

In [ ]:
# display as a bar chart
plt.bar(modelNames,scores)
plt.ylim(0.8, 1.0)
plt.show()